# Create virtual machines in Orbstack(MacOS)/ multipass(Ubuntu)

This setup is meant for running Ansible playbooks on MacOS/Ubuntu. 

## Prerequisites for MacOs
1. Install [OrbStack](orbstack.dev)
2. Confirm `orb` command is now available on your command line
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/intro_installation.html)

## Prerequisites for Ubuntu
1. Install [multipass](https://canonical.com/multipass/install)
2. Confirm installation with `multipass --version` command
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/installation_distros.html#installing-ansible-on-debian)

Either run the commands manually from command like (for live output) or by executing cells here

Identifying the OS you are currently using

In [ ]:
import platform

linux = platform.system() == "Linux"

Running the provision script according to OS

In [ ]:
import subprocess
if linux:
    subprocess.run(["bash", "./provision-linux.sh"], check=True)
else:
    subprocess.run(["bash", "./provision-mac.sh"], check=True)


Notice that the following task will prompt (in VSCode) for your dockerhub username and password

In [ ]:
import os

os.environ["DOCKER_USERNAME"] = input("Enter Dockerhub username: ")
os.environ["DOCKER_PASSWORD"] = input("Enter Dockerhub password: ")

The next command is run with Python instead of %%bash simply because %%bash doesn't support real-time output

In [ ]:
inventory = 'local.macos.yml'
if linux:
    inventory = 'local.linux.yml'

cmd = f"""
mongo_user=$(openssl rand -hex 8)
mongo_pass=$(openssl rand -base64 24)
backup_pass=$(openssl rand -base64 16)
es_pass=$(openssl rand -base64 24)

ansible-playbook -i {inventory} ../server-setup/playbook.yml \\
  -e encrypted_disk_size=200g \\
  -e dockerhub_username=$DOCKER_USERNAME \\
  -e dockerhub_password=$DOCKER_PASSWORD \\
  -e mongodb_admin_username=$mongo_user \\
  -e mongodb_admin_password="$mongo_pass" \\
  -e backup_encryption_passphrase="$backup_pass" \\
  -e elasticsearch_superuser_password="$es_pass" \\
  --private-key ./.ssh/ssh-key
  -vv
"""

In [ ]:
import subprocess

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True, env=os.environ)
for line in proc.stdout:
    print(line, end="")
proc.wait()